In [11]:
!pip install -qU elasticsearch openai


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [18]:
## Install the required packages
## pip install -qU elasticsearch openai
import os
from elasticsearch import Elasticsearch
from openai import AzureOpenAI




es_client = Elasticsearch(
    "https://d9ebba58dda54e92a84b430cfe6ad6a8.us-east-1.aws.found.io:443",
    api_key=os.getenv("ES_API_KEY", "N01nQW1KSUJULThfSWlmUGd4aHQ6ZmU0WFI4a2FUOGloeHBZSFpsMlZQZw==")
)

endpoint = os.getenv("ENDPOINT_URL", "https://openai-test-muhammad.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "muhammad_test")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "fb91a3bb827345f6bef647897e63c9f3")

#"ENDPOINT_URL","https://openai-test-muhammad.openai.azure.com/openai/deployments/muhammad_test/chat/completions?api-version=2024-02-15-preview"
openai_client = AzureOpenAI(
    #api_key=os.getenv("OPENAI_API_KEY","fb91a3bb827345f6bef647897e63c9f3"),
    #base_url=os.getenv("ENDPOINT_URL","https://openai-test-muhammad.openai.azure.com")
    api_version="2024-02-15-preview",
)
index_source_fields = {
    "workplace-app-docs": [
        "text"
    ]
}
def get_elasticsearch_results(query):
    es_query = {
        "retriever": {
            "standard": {
                "query": {
                    "text_expansion": {
                        "vector.tokens": {
                            "model_id": ".elser_model_2",
                            "model_text": query
                        }
                    }
                }
            }
        },
        "size": 3
    }
    result = es_client.search(index="workplace-app-docs", body=es_query)
    return result["hits"]["hits"]
def create_openai_prompt(results):
    context = ""
    for hit in results:
        inner_hit_path = f"{hit['_index']}.{index_source_fields.get(hit['_index'])[0]}"
        ## For semantic_text matches, we need to extract the text from the inner_hits
        if 'inner_hits' in hit and inner_hit_path in hit['inner_hits']:
            context += '\n --- \n'.join(inner_hit['_source']['text'] for inner_hit in hit['inner_hits'][inner_hit_path]['hits']['hits'])
        else:
            source_field = index_source_fields.get(hit["_index"])[0]
            hit_context = hit["_source"][source_field]
            context += f"{hit_context}\n"
    prompt = f"""
  Instructions:

  - You are an assistant for question-answering tasks.
  - Answer questions truthfully and factually using only the context presented.
  - If you don't know the answer, just say that you don't know, don't make up an answer.
  - You must always cite the document where the answer was extracted using inline academic citation style [], using the position.
  - Use markdown format for code examples.
  - You are correct, factual, precise, and reliable.

  Context:
  {context}

  """
    return prompt
def generate_openai_completion(user_prompt, question):
    response = openai_client.completions.create(
        model="muhammad_test",
        prompt="Test prompt"
        # messages=[
        #     {"role": "system", "content": user_prompt},
        #     {"role": "user", "content": question},
        # ]
    )
    return response.choices[0].message.content


ValueError: Must provide either the `api_version` argument or the `OPENAI_API_VERSION` environment variable

In [19]:
if __name__ == "__main__":
    question = "how NASA sales work"
    elasticsearch_results = get_elasticsearch_results(question)
    context_prompt = create_openai_prompt(elasticsearch_results)
    openai_completion = generate_openai_completion(context_prompt, question)
    print(openai_completion)

/var/folders/xb/k21g97617gs5lc1g4wh0452m0000gn/T/ipykernel_40207/1411945610.py:36: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  result = es_client.search(index="workplace-app-docs", body=es_query)


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [23]:
#%pip install -qU langchain-openai

!pip uninstall langchain-openai
!pip install langchain-openai

Found existing installation: langchain-openai 0.2.2
Uninstalling langchain-openai-0.2.2:
  Would remove:
    /Users/msarwar/workspace/elasticsearch-labs/example-apps/chatbot-rag-app/venv/lib/python3.10/site-packages/langchain_openai-0.2.2.dist-info/*
    /Users/msarwar/workspace/elasticsearch-labs/example-apps/chatbot-rag-app/venv/lib/python3.10/site-packages/langchain_openai/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
  Using cached langchain_core-0.3.11-py3-none-any.whl (407 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.5
    Uninstalling langchain-core-0.3.5:
      Successfully uninstalled langchain-core-0.3.5

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [24]:




from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="muhammad_test",  # or your deployment
    api_version="2024-02-15-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

ImportError: cannot import name 'InputTokenDetails' from 'langchain_core.messages.ai' (/Users/msarwar/workspace/elasticsearch-labs/example-apps/chatbot-rag-app/venv/lib/python3.10/site-packages/langchain_core/messages/ai.py)